# README
## TLDR;
* Dear, beginner
* who want to explore EDA from basic
* who want to do Data Analysis process from scratch
* who are beginner like me :)

![](https://preview.ibb.co/bKsv9k/k.jpg)
# INTRODUCTION
* 본 노트북은 <a href="https://www.kaggle.com/kanncaa1/feature-selection-and-data-visualization">Feature Selection and Data Visualization</a> 노트북을 번역한 노트북입니다
    * Kaggle Notebook Grandmaster의 노트북을 필사하며 데이터 분석의 기본을 다질 수 있습니다
    * 데이터 분석을 우리가 좋아하는 pub(🍺)에 비유하는 등 분석의 재미를 느낄 수 있습니다!
    
    
* 본 노트북에서는 다른 노트북과 달리 feature visualization과 selection에 초점을 맞춥니다
    * Visualization
        * Violin plot, Joint plot, pair grid, swarm plot, heatmap 등을 통해 데이터에서 insight를 얻습니다
    * feature selection
        * correlation, univariate feature selection, recursive feature elimination, recursive feature elimination with cross validation, random forest와 함께 tree based feature selection을 사용합니다


* 한편 주성분 분석을 위해 PCA를 사용합니다 

> **Enjoy your data analysis!!!**<br>
> 본 노트북을 통해 데이터 분석을 즐기시길 바랍니다!



# Data Analysis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns # data visualization library
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory
# e.g. running the below will list the files in the input directory
import time
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
data = pd.read_csv('../input/data.csv')

> Before making anything like feature selection,feature extraction and classification, firstly we start with basic data analysis. <br>
Lets look at features of data.<br>

🖐️  Notice! (시작 전!)<br>
* feature selection, feature extraction, classification을 하기 전에 해야 할 것은 무엇일까?
* feature selection, feature extraction, classification을 하기 전에 우선 데이터의 feature부터 살펴보겠습니다!


In [ ]:
data.head()  # head method show only first 5 rows

데이터를 한 번 살펴보자. 어떤 insight를 얻을 수 있을까?

**There are 4 things that take my attention**

1) There is an **id** that cannot be used for classificaiton<br>
2) **Diagnosis** is our class label<br>
3) **Unnamed: 32** feature includes NaN so we do not need it.<br>
4) I do not have any idea about other feature names actually I do not need because machine learning is awesome **:)<br>**


Therefore, drop these unnecessary features. However do not forget this is not a feature selection. This is like a browse a pub, we do not choose our drink yet !!!


1) 분류에서는 사용되지 않는 id feature가 있다<br>
2) Diagnosis는 우리의 target label이다<br>
3) Unnamed: 32 feature는 NaN값만 포함하고 있다. 분석 단계에서는 필요없을 것 같다<br>
4) 다른 feature들에 대해서는 하나씩 살펴볼 필요가 있을 것 같다

In [ ]:
# feature names as a list
col = data.columns
print(col)

In [ ]:
# y includes our labels and x includes our features
y = data.diagnosis
list = ['id', 'diagnosis', 'Unnamed: 32']
x = data.drop(list, axis=1)
x.head()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))

data.diagnosis.value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0])
sns.countplot('diagnosis', data=data, order=data['diagnosis'].value_counts().index, ax=ax[1])
ax[0].legend()
ax[1].legend()
B, M = y.value_counts()
print("Number of Benign: ", B)
print("Number of Malignant: ", M)

* 이제 우리는 feature를 갖게 되었다. 그런데 저러한 feature들이 무엇을 의미하는지 어떻게 알 수 있을까? feature들에 대해 얼마나 알아야 할까?
    * 정답은 모든 feature에 대해 알 필요는 없다는 것이다
    * 대신에 feature의 variance, stardard deviation, number of sample 또는 min/max value 등에 대해서는 반드시 알아야 한다
* variance, stardard deviation, number of sample, min/max 등의 정보는 데이터에 대해 이해하는 것을 도와준다
    * Pandas Dataframe에서는 친절하게도 .describe()메서드를 통해 데이터에 대한 통계치를 제공해준다


In [ ]:
x.describe()

* area_mean과 smoothness_mean을 보면 최대값이 각각 2501과 0.163400으로 크게 차이나는 것을 확인할 수 있다
* 따라서 시각화와 feature selection, feature extraction, classification을 위해서는 standardization과 normalization을 해줘야 한다

* Standardization vs Normalization
    * > The terms normalization and standardization are sometimes used interchangeably, but they usually refer to different things. Normalization usually means to scale a variable to have a values between 0 and 1, while standardization transforms data to have a mean of zero and a standard deviation of 1.
    * by Googling
    * Standardization은 데이터가 mean 0, standard deviation 1을 갖도록 데이터를 변형해주는 것입니다
    * 반면 Normalization은 변수 값의 범위를 0에서 1사이로 스케일 해주는 것입니다

# Visualization

* seaborn plot을 사용해 시각화를 할 것이다
* 시각화 단계에서 주의할 점은 '시각화'와 'feature selection'이 다르다는 것이다

Before violin and swarm plot we need to normalization or standirdization. Because differences between values of features are very high to observe on plot. I plot features in 3 group and each group includes 10 features to observe better.


* violin plot과 swarm plot을 그리기 전에 normalization과 standardization을 해줘야 한다
    * 왜 해야할까?
        * normalization을 하지 않고 plot을 그려보면 normalization과 standardization이 '필수'라는 사실을 확인할 수 있을 것이다

* normalization을 하기 전/후로 비교해보겠습니다

In [ ]:
data_dia = y
data = x
# don't nomalize features

data = pd.concat([data_dia, data.iloc[:, 0:10]], axis=1)
print(data.shape)

data = pd.melt(data, id_vars="diagnosis",
                      var_name="features",
                      value_name="value")
print(data.head())

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data, split=True, inner="quart")
plt.xticks(rotation=90)

* 위의 도표에서 볼 수 있는 것처럼 normalization을 하지 않으면 시각화를 통해 데이터를 파악하기가 매우 어렵다
* 이제 noramization을 한 뒤에 graph를 해석해보자

In [ ]:
# first ten features
data_dia = y
data = x
data_n_2 = (data - data.mean()) / data.std()  # pd.std(): return sample standardization over requested axis

data = pd.concat([data_dia, data_n_2.iloc[:, 0:10]], axis=1)
print(data.shape)
# data.head()
data = pd.melt(data, id_vars="diagnosis",
                      var_name="features",
                      value_name="value")
print(data.head())
print(data.shape)

In [ ]:
plt.figure(figsize=(10, 10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data, split=True, inner="quart")
plt.xticks(rotation=90)

* 해석해보자
    * testure_mean에서 Malignant(악성)과 Benign의 median(중위값)은 분류되는 것 처럼 보인다. 해당 feature를 분류기를 만들때 고려하면 좋을 것 같다
    * 반면 fractal_dimension_mean feature에서 Malignant와 Benign의 median(중위값)은 악성과 정상을 구분하지 못하는 것처럼 보인다
 
* 다른 feature들도 살펴보도록 한다

In [ ]:
# second ten features
data = pd.concat([y, data_n_2.iloc[:, 10:20]], axis=1)
data = pd.melt(data, id_vars="diagnosis",
                      var_name="features",
                      value_name="value")
plt.figure(figsize=(10, 10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data, split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# Third ten features
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# As an alternative of violin plot, box plot can be used
# box plots are also useful in terms of seeing outliers
# I do not visualize all features with box plot
# In order to show you lets have an example of box plot
# If you want, you can visualize other features as well.


# violin plot의 대체재로 box plotdl 사용될 수 있다
plt.figure(figsize=(10, 10))
sns.boxplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

* 도표를 해석해보자
    * box plot을 보면 concavity_worst와 concave point_worst가 비슷해보인다
    * 그렇다면 실제로 상관관계가 있는지 어떻게 알 수 있을까?
    * joint plot을 통해 feature를 조금 더 깊게 살펴보자

In [ ]:
sns.jointplot(x.loc[:, 'concavity_worst'], x.loc[:, 'concave points_worst'], kind="regg", color="#ce1414")

* joint plot을 통해 살펴본 결과 해당 feature들이 실제로 상관관계가 있는 것을 확인할 수 있었다
* pearson value는 correlation 값으로 1이면 높은 양의 상관관계를 의미한다
    * 해당 도표에서 볼 수 있듯이 pearson value가 0.86으로 매우 높은 것을 확인할 수 있다

> 그렇다면 3개 이상의 변수는 어떻게 비교할 수 있을까?

In [ ]:
sns.set(style="white")
df = x.loc[:, ['radius_worst', 'perimeter_worst', 'area_worst']]
g = sns.PairGrid(df, diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")  # 아래쪽에는 KDE plot을 그려준다
g.map_upper(plt.scatter)  # 위쪽에는 scatter plot을 그려준다
g.map_diag(sns.kdeplot, lw=3)

* pair plot을 살펴보면 세 변수가 높은 상관관계가 있다는 것을 확인할 수 있다

Up to this point, we make some comments and discoveries on data already. If you like what we did, I am sure swarm plot will open the pub's door :) 

In swarm plot, I will do three part like violin plot not to make plot very complex appearance

### swarm plot
* Draw a categorical scatterplot with non-overlapping points.
* categorical scatter plot을 포인트가 겹치지 않게 그려줍니다
    * 여기서 포인트가 겹치지 않게 그려준다 함을 다시 한번 생각해보시면 좋을 것 같습니다!

In [ ]:
sns.set(style="whitegrid", palette="muted")
data_dia = y
data = x
data_n_2 = (data-data.mean()) / (data.std())  # standardization

data = pd.concat([y, data_n_2.iloc[:, 0:10]], axis=1)
data = pd.melt(data, id_vars='diagnosis',
                      var_name="features",
                      value_name='value')
plt.figure(figsize=(10, 10))
tic = time.time()
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,10:20]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
toc = time.time()
plt.xticks(rotation=90)
print("swarm plot time: ", toc-tic ," s")

They looks cool right. And you can see variance more clear. Let me ask you a question, **in these three plots which feature looks like more clear in terms of classification.** In my opinion **area_worst** in last swarm plot looks like malignant and benign are seprated not totaly but mostly. Hovewer, **smoothness_se** in swarm plot 2 looks like malignant and benign are mixed so it is hard to classfy while using this feature.

* variance를 보다 선명하게 관측할 수 있게 되었다
* 마지막 plot의 area_worst는 malignant와 benign을 대부분 분류해주는 것을 볼 수 있다
* 두 번째 plot의 smoothness_se의 경우 malignant와 benign을 제대로 분류하지 못하는 것을 확인할 수 있다

**What if we want to observe all correlation between features?** Yes, you are right. The answer is heatmap that is old but powerful plot method.

In [ ]:
# correlation map
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x.corr(), annot=True, linewidth=.5, fmt='.1f', ax=ax)

Well, finaly we are in the pub and lets choose our drinks at feature selection part while using heatmap(correlation matrix).

> 드디어 pub에 도착했다

* 🍺  우리가 원하는 드링크를 고르자!
* 'feature selection'을 하자는 말이다!

# Feature Selection and Random Forest Classification

>Today our purpuse is to try new cocktails. For example, we are finaly in the pub(🍺) and we want to drink different tastes. Therefore, we need to compare ingredients of drinks. If one of them includes lemon, after drinking it we need to eliminate other drinks which includes lemon so as to experience very different tastes.

* 오늘 우리는 pub에서 새로운 cocktail을 시도해볼 것이다
    * 다른 종류의 drink를 시도할 것이다
    * drink의 성분을 비교해서 lemen을 먹었으면 다음 음료로는 lemon이 제외된 음료를 시키는 식으로 말이다

> In this part we will select feature with different methods that are feature selection with correlation, univariate feature selection, recursive feature elimination (RFE), recursive feature elimination with cross validation (RFECV) and tree based feature selection. We will use random forest classification in order to train our model and predict. 

* 이번 파트에서는 다른 방법들로부터 feature를 선택할 것이다
  * correlation, univariate feature selection, recursive feature elimination(RFE), recursive feature elimination with cross validation(EFECV), tree based feature selection 등을 알아보자
  * 모델의 학습은 random forest classifier를 사용할 것이다

## 1) Feature selection with correlation and random forest classification

* heatmap을 보면 raidus_mean feature와 perimeter_mean, area_mean은 높은 양의 상관관계를 보여주고 있다
* 이중에서 필자는 radius_mean을 직관적으로 선택했다고 한다
    * Why?
        * swarm plot을 살펴보면 radius_mean feature가 clear하게 malignant와 benign을 구분해줘서라고 한다
        * (내 생각엔) perimeter_mean과 area_mean도 마찬가지인 것 같은데...ㅎㅎ


* 몇 개의 feature를 더 살펴보면...
    * Compactness_mean과 concavity_mean, concave points_mean에서 concavity_mean을 선택 (swarm plot)을 보자
    * radius_se, perimeter_se, area_se로부터 area_se 선택 (swarm plot을 보자)
    * radius_wrost, perimeter_worst, area_worst에서 area_worst 선택
    * concavity_worst, concavity_worst, concave point_worst에서 oncavity_worst 선택
    * ...


In [ ]:
drop_list1 = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se','radius_worst','perimeter_worst','compactness_worst','concave points_worst','compactness_se','concave points_se','texture_worst','area_worst']
x_1 = x.drop(drop_list1,axis = 1 )        # do not modify x, we will use it later 
x_1.head()


After drop correlated features, as it can be seen in below correlation matrix, there are no more correlated features. Actually, I know and you see there is correlation value 0.9 but lets see together what happen if we do not drop it.

* correlation heatmap을 통해 상관관계가 높은 feature들을 제거해주었다
* 상관관계가 높은 feature들을 제거해준 뒤 다시 heatmap을 그려보자

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(14, 14))
sns.heatmap(x_1.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

Well, we choose our features but **did we choose correctly ?** Lets use random forest and find accuracy according to chosen features.

* 우리는 상관관계가 높은 feature들을 제거해주는 방식으로 feature를 선택해주었다 (feature selection)
* 이러한 방법이 효과적인지 random forest 분류기를 통해 확인해보자

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score

# split data train 70% and test 30%
x_train, x_test, y_train, y_test = train_test_split(x_1, y, test_size=0.3, random_state=42)

# random forest classifier with n_esitimate=10(default)
clf_rf = RandomForestClassifier(random_state=43)
clf_rf = clf_rf.fit(x_train, y_train)

ac = accuracy_score(y_test, clf_rf.predict(x_test))
print("Accuracy is: ", ac)
cm = confusion_matrix(y_test, clf_rf.predict(x_test))
sns.heatmap(cm, annot=True, fmt="d")

Accuracy is almost 95% and as it can be seen in confusion matrix, we make few wrong prediction. 
Now lets see other feature selection methods to find better results.

## 2) Univariate feature selection and random forest classification
In univariate feature selection, we will use SelectKBest that removes all but the k highest scoring features.
<http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest>

> In this method we need to choose how many features we will use. For example, will k (number of features) be 5 or 10 or 15? The answer is only trying or intuitively. I do not try all combinations but I only choose k = 5 and find best 5 features.

* correlation이 높은 feature를 제거하는 feature selection에 이어 다른 feature selection 방법에 대해 알아보자
* Univariate feature selection
    * 'univariable analysis' 두 변수 사이의 관계를 밝히는 방법
    * SelctKBest를 사용한다
    * "Select features according to the k highest scores"
    * k개의 높은 스코어를 갖는 feature를 선택한다
    * 이곳 <a href='https://datascienceschool.net/03%20machine%20learning/14.03%20%ED%8A%B9%EC%A7%95%20%EC%84%A0%ED%83%9D.html'>link</a>에서 자세한 내용을 알아볼 수 있다

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(x_train, y_train)

In [ ]:
print("Score list: ", select_feature.scores_)  # scores of features
print("Feature list: ", x_train.columns)

Best 5 feature to classify is that **area_mean, area_se, texture_mean, concavity_worst and concavity_mean**. So lets se what happens if we use only these best scored 5 feature.

In [ ]:
x_train_2 = select_feature.transform(x_train)
x_test_2 = select_feature.transform(x_test)

# random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()
clf_rf_2 = clf_rf_2.fit(x_train_2, y_train)
ac_2 = accuracy_score(y_test, clf_rf_2.predict(x_test_2))
print("Accuracy is: ", ac_2)
cm_2 = confusion_matrix(y_test, clf_rf_2.predict(x_test_2))
sns.heatmap(cm_2, annot=True, fmt="d")

* 성능이 좋은 변수를 10개 추출한다면 모델의 성능이 더 좋아질까?

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# find best scored 5 features
select_feature = SelectKBest(chi2, k=10).fit(x_train, y_train)

In [ ]:
x_train_2 = select_feature.transform(x_train)
x_test_2 = select_feature.transform(x_test)
# random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()
clf_rf_2 = clf_rf_2.fit(x_train_2, y_train)
ac_2 = accuracy_score(y_test, clf_rf_2.predict(x_test_2))
print("Accuracy is: ", ac_2)
# also print confusion matrix
cm_2 = confusion_matrix(y_test, clf_rf_2.predict(x_test_2))
sns.heatmap(cm_2, annot=True, fmt="d")

* 성능이 더 좋아졌다!
* 하이퍼 파라미터 서치를 통해 가장 좋은 성능의 모델을 만들수도 있을 것 같다
    * 하이퍼 파라미터 서치도 한 번 연습해보자


Accuracy is almost 96% and as it can be seen in confusion matrix, we make few wrong prediction. What we did up to now is that we choose features according to correlation matrix and according to selectkBest method. Although we use 5 features in selectkBest method accuracies look similar.
Now lets see other feature selection methods to find better results.

## 3) Recursive feature elimination (RFE) with random forest
> <http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html>
Basically, it uses one of the classification methods (random forest in our example), assign weights to each of features. Whose absolute weights are the smallest are pruned from the current set features. That procedure is recursively repeated on the pruned set until the desired number of features

> Like previous method, we will use 5 features. However, which 5 features will we use ? We will choose them with RFE method.

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()
rfe = RFE(estimator=clf_rf_3, n_features_to_select=5, step=1)
rfe = rfe.fit(x_train, y_train)

In [ ]:
print("Chosen best 5 feature by rfe: ", x_train.columns[rfe.support_])

* Modeling it!

In [ ]:
x_train_3 = x_train.loc[:, x_train.columns[rfe.support_]]
x_test_3 = x_test.loc[:, x_test.columns[rfe.support_]]

clf_rf_3 = RandomForestClassifier()
clf_rf_3.fit(x_train_3, y_train)

ac_3 = accuracy_score(y_test, clf_rf_3.predict(x_test_3))
print("Accuracy: ", ac_3)
# confusion matrics
cm_3 = confusion_matrix(y_test, clf_rf_3.predict(x_test_3))
sns.heatmap(cm_3, annot=True, fmt="d")

* Chosen 5 best features by rfe is **texture_mean, area_mean, concavity_mean, area_se, concavity_worst**. 
* They are exactly similar with previous (selectkBest) method. Therefore we do not need to calculate accuracy again. 
* Shortly, we can say that we make good feature selection with rfe and selectkBest methods. However as you can see there is a problem, okey I except we find best 5 feature with two different method and these features are same but why it is **5**. 
* Maybe if we use best 2 or best 15 feature we will have better accuracy. 
* Therefore lets see how many feature we need to use with rfecv method.

* selectKBest mothod와 RFE를 통해 5개의 feature를 추출할 수 있었다
* 두 방법 모두 같은 feature를 추출해줬으며 RandomForest Classifier를 통해 95%의 성능을 달성할 수 있었다
* 하지만 여기에는 한 가지 문제점이 존재한다
    * 5개의 feature가 최적의 feature라고 할 수 있을까?
    * rfecv 방법을 통해 최적의 feature를 추출해보자!

## 4) Recursive feature elimination with cross validation and random forest classification
> <http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html>
Now we will not only **find best features** but we also find **how many features do we need** for best accuracy.

* 이제는 최적의 feature뿐만아니라 몇 개의 feature가 필요한지도 알아보도록 하자
* 최고의 성능을 위한 feature의 개수는 몇 개인지 알아보자!

In [ ]:
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct classifications
clf_rf_4 = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(x_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

Finally, we find best 11 features that are **texture_mean, area_mean, concavity_mean, texture_se, area_se, concavity_se, symmetry_se, smoothness_worst, concavity_worst, symmetry_worst and fractal_dimension_worst** for best classification. Lets look at best accuracy with plot.


In [ ]:
# Plot number of features vs cross-validation scores
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score of number of selected feature")
plt.plot(range(1, len(rfecv.grid_scores_) +1), rfecv.grid_scores_)
plt.show()    

Lets look at what we did up to this point. Lets accept that guys this data is very easy to classification. However, our first purpose is actually not finding good accuracy. Our purpose is learning how to make **feature selection and understanding data.** Then last make our last feature selection method.

## 5) Tree based feature selection and random forest classification
> <http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html>
In random forest classification method there is a **feature_importances_** attributes that is the feature importances (the higher, the more important the feature). **!!! To use feature_importance method, in training data there should not be correlated features. Random forest choose randomly at each iteration, therefore sequence of feature importance list can change.**


* tree 기반의 feature selection과 randomforest classifier
* randomforest classifier에서 제공하는 feature_importance 방법을 사용하려면 학습 데이터에 상관관계가 있는 feature가 있어서는 안된다


In [ ]:
clf_rf_5 = RandomForestClassifier()      
clr_rf_5 = clf_rf_5.fit(x_train,y_train)
importances = clr_rf_5.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf_rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest

plt.figure(1, figsize=(14, 13))
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances[indices],
       color="g", yerr=std[indices], align="center")
plt.xticks(range(x_train.shape[1]), x_train.columns[indices],rotation=90)
plt.xlim([-1, x_train.shape[1]])
plt.show()

As you can seen in plot above, after 5 best features importance of features decrease. Therefore we can focus these 5 features. As I sad before, I give importance to understand features and find best of them. 

# Feature Extraction
> <http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html>
We will use principle component analysis (PCA) for feature extraction. Before PCA, we need to normalize data for better performance of PCA.
 

* feature extraction을 위해 PCA를 사용할 것이다
* PCA의 성능을 높이기 위해서는 데이터를 normalize 해줘야 한다

In [ ]:
# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
#normalization
x_train_N = (x_train-x_train.mean())/(x_train.max()-x_train.min())
x_test_N = (x_test-x_test.mean())/(x_test.max()-x_test.min())

from sklearn.decomposition import PCA
pca = PCA()
pca.fit(x_train_N)

plt.figure(1, figsize=(14, 13))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_ratio_, linewidth=2)
plt.axis('tight')
plt.xlabel('n_components')
plt.ylabel('explained_variance_ratio_')

According to variance ration, 3 component can be chosen.

# Conclusion
* 본 노트북에서는 feature selection과 data visualization을 상세히 다뤘습니다
* 처음 우리가 맞이한 데이터는 33개의 feature를 가지고 있었지만 feature selection을 통해 feature의 수를 33개에서 5개까지 줄일 수 있었습니다. 이 때의 정확도는 95%에 달했습니다
* 오늘 배운 방법으로 데이터 분석을 시작하는 계기가 되었으면 좋겠습니다. 감사합니다!